## Text Generation with Neural Networks

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
tf.__version__

'2.3.0'

## Step 1: The Data

You can grab any free text you want from here: https://www.gutenberg.org/

We'll choose all of shakespeare's works (which we have already downloaded for you), mainly for two reasons:

1. Its a large corpus of text, its usually recommended you have at least a source of 1 million characters total to get realistic text generation.

2. It has a very distinctive style. Since the text data uses old style english and is formatted in the style of a stage play, it will be very obvious to us if the model is able to reproduce similar results.

In [3]:
path_to_file = '/content/drive/My Drive/Colab Notebooks/shakespeare.txt'

In [4]:
text = open(path_to_file, 'r').read()

In [5]:
text[:500]

"\n                     1\n  From fairest creatures we desire increase,\n  That thereby beauty's rose might never die,\n  But as the riper should by time decease,\n  His tender heir might bear his memory:\n  But thou contracted to thine own bright eyes,\n  Feed'st thy light's flame with self-substantial fuel,\n  Making a famine where abundance lies,\n  Thy self thy foe, to thy sweet self too cruel:\n  Thou that art now the world's fresh ornament,\n  And only herald to the gaudy spring,\n  Within thine own bu"

In [6]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


### Understanding unique characters

In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print(vocab)
len(vocab)

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '}']


84

In [8]:
type(vocab)

list

## Step 2: Text Processing

### Text Vectorization

We know a neural network can't take in the raw string data, we need to assign numbers to each character. Let's create two dictionaries that can go from numeric index to character and character to numeric index.

In [9]:
for pair in enumerate(vocab[:10]):
    print(pair)

(0, '\n')
(1, ' ')
(2, '!')
(3, '"')
(4, '&')
(5, "'")
(6, '(')
(7, ')')
(8, ',')
(9, '-')


In [10]:
char_to_ind = {u:i for i, u in enumerate(vocab)}

In [11]:
char_to_ind

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '&': 4,
 "'": 5,
 '(': 6,
 ')': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '4': 15,
 '5': 16,
 '6': 17,
 '7': 18,
 '8': 19,
 '9': 20,
 ':': 21,
 ';': 22,
 '<': 23,
 '>': 24,
 '?': 25,
 'A': 26,
 'B': 27,
 'C': 28,
 'D': 29,
 'E': 30,
 'F': 31,
 'G': 32,
 'H': 33,
 'I': 34,
 'J': 35,
 'K': 36,
 'L': 37,
 'M': 38,
 'N': 39,
 'O': 40,
 'P': 41,
 'Q': 42,
 'R': 43,
 'S': 44,
 'T': 45,
 'U': 46,
 'V': 47,
 'W': 48,
 'X': 49,
 'Y': 50,
 'Z': 51,
 '[': 52,
 ']': 53,
 '_': 54,
 '`': 55,
 'a': 56,
 'b': 57,
 'c': 58,
 'd': 59,
 'e': 60,
 'f': 61,
 'g': 62,
 'h': 63,
 'i': 64,
 'j': 65,
 'k': 66,
 'l': 67,
 'm': 68,
 'n': 69,
 'o': 70,
 'p': 71,
 'q': 72,
 'r': 73,
 's': 74,
 't': 75,
 'u': 76,
 'v': 77,
 'w': 78,
 'x': 79,
 'y': 80,
 'z': 81,
 '|': 82,
 '}': 83}

In [12]:
char_to_ind['H']

33

In [13]:
ind_to_char = np.array(vocab)

In [14]:
ind_to_char

array(['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i',
       'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
       'w', 'x', 'y', 'z', '|', '}'], dtype='<U1')

In [15]:
type(ind_to_char)

numpy.ndarray

In [19]:
ind_to_char[33]

'H'

In [ ]:
#list(char_to_ind[c] for c in text)[:10]

In [17]:
encoded_text = np.array([char_to_ind[c] for c in text])

In [18]:
encoded_text

array([ 0,  1,  1, ..., 30, 39, 29])

In [20]:
encoded_text.shape

(5445609,)

We now have a mapping we can use to go back and forth from characters to numerics.

In [21]:
sample = text[:500]
sample

"\n                     1\n  From fairest creatures we desire increase,\n  That thereby beauty's rose might never die,\n  But as the riper should by time decease,\n  His tender heir might bear his memory:\n  But thou contracted to thine own bright eyes,\n  Feed'st thy light's flame with self-substantial fuel,\n  Making a famine where abundance lies,\n  Thy self thy foe, to thy sweet self too cruel:\n  Thou that art now the world's fresh ornament,\n  And only herald to the gaudy spring,\n  Within thine own bu"

In [22]:
encoded_text[:500]

array([ 0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1, 12,  0,  1,  1, 31, 73, 70, 68,  1, 61, 56, 64,
       73, 60, 74, 75,  1, 58, 73, 60, 56, 75, 76, 73, 60, 74,  1, 78, 60,
        1, 59, 60, 74, 64, 73, 60,  1, 64, 69, 58, 73, 60, 56, 74, 60,  8,
        0,  1,  1, 45, 63, 56, 75,  1, 75, 63, 60, 73, 60, 57, 80,  1, 57,
       60, 56, 76, 75, 80,  5, 74,  1, 73, 70, 74, 60,  1, 68, 64, 62, 63,
       75,  1, 69, 60, 77, 60, 73,  1, 59, 64, 60,  8,  0,  1,  1, 27, 76,
       75,  1, 56, 74,  1, 75, 63, 60,  1, 73, 64, 71, 60, 73,  1, 74, 63,
       70, 76, 67, 59,  1, 57, 80,  1, 75, 64, 68, 60,  1, 59, 60, 58, 60,
       56, 74, 60,  8,  0,  1,  1, 33, 64, 74,  1, 75, 60, 69, 59, 60, 73,
        1, 63, 60, 64, 73,  1, 68, 64, 62, 63, 75,  1, 57, 60, 56, 73,  1,
       63, 64, 74,  1, 68, 60, 68, 70, 73, 80, 21,  0,  1,  1, 27, 76, 75,
        1, 75, 63, 70, 76,  1, 58, 70, 69, 75, 73, 56, 58, 75, 60, 59,  1,
       75, 70,  1, 75, 63

## Step 3: Creating Batches

Overall what we are trying to achieve is to have the model predict the next highest probability character given a historical sequence of characters. Its up to us (the user) to choose how long that historic sequence. Too short a sequence and we don't have enough information (e.g. given the letter "a" , what is the next character) , too long a sequence and training will take too long and most likely overfit to sequence characters that are irrelevant to characters farther out. While there is no correct sequence length choice, you should consider the text itself, how long normal phrases are in it, and a reasonable idea of what characters/words are relevant to each other.

In [23]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [24]:
line = "From fairest creatures we desire increase"

In [25]:
len(line)

41

In [26]:
part_stanza = '''From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,'''

In [27]:
len(part_stanza)

131

### Training Sequences

The actual text data will be the text sequence shifted one character forward. For example:

Sequence In: "Hello my nam"
Sequence Out: "ello my name"


We can use the `tf.data.Dataset.from_tensor_slices` function to convert a text vector into a stream of character indices.

In [28]:
seq_len = 120

In [29]:
total_num_seq = len(text)//(seq_len+1)

In [30]:
total_num_seq

45005

In [31]:
# Create Training Sequences
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

for i in char_dataset.take(500):
     print(ind_to_char[i.numpy()])



 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
1


 
 
F
r
o
m
 
f
a
i
r
e
s
t
 
c
r
e
a
t
u
r
e
s
 
w
e
 
d
e
s
i
r
e
 
i
n
c
r
e
a
s
e
,


 
 
T
h
a
t
 
t
h
e
r
e
b
y
 
b
e
a
u
t
y
'
s
 
r
o
s
e
 
m
i
g
h
t
 
n
e
v
e
r
 
d
i
e
,


 
 
B
u
t
 
a
s
 
t
h
e
 
r
i
p
e
r
 
s
h
o
u
l
d
 
b
y
 
t
i
m
e
 
d
e
c
e
a
s
e
,


 
 
H
i
s
 
t
e
n
d
e
r
 
h
e
i
r
 
m
i
g
h
t
 
b
e
a
r
 
h
i
s
 
m
e
m
o
r
y
:


 
 
B
u
t
 
t
h
o
u
 
c
o
n
t
r
a
c
t
e
d
 
t
o
 
t
h
i
n
e
 
o
w
n
 
b
r
i
g
h
t
 
e
y
e
s
,


 
 
F
e
e
d
'
s
t
 
t
h
y
 
l
i
g
h
t
'
s
 
f
l
a
m
e
 
w
i
t
h
 
s
e
l
f
-
s
u
b
s
t
a
n
t
i
a
l
 
f
u
e
l
,


 
 
M
a
k
i
n
g
 
a
 
f
a
m
i
n
e
 
w
h
e
r
e
 
a
b
u
n
d
a
n
c
e
 
l
i
e
s
,


 
 
T
h
y
 
s
e
l
f
 
t
h
y
 
f
o
e
,
 
t
o
 
t
h
y
 
s
w
e
e
t
 
s
e
l
f
 
t
o
o
 
c
r
u
e
l
:


 
 
T
h
o
u
 
t
h
a
t
 
a
r
t
 
n
o
w
 
t
h
e
 
w
o
r
l
d
'
s
 
f
r
e
s
h
 
o
r
n
a
m
e
n
t
,


 
 
A
n
d
 
o
n
l
y
 
h
e
r
a
l
d
 
t
o
 
t
h
e
 
g
a
u
d
y
 
s
p
r
i
n
g
,


 
 
W
i
t
h
i
n
 
t
h
i
n
e
 
o
w
n
 
b
u


The **batch** method converts these individual character calls into sequences we can feed in as a batch. We use seq_len+1 because of zero indexing. Here is what drop_remainder means:

drop_remainder: (Optional.) A `tf.bool` scalar `tf.Tensor`, representing
    whether the last batch should be dropped in the case it has fewer than
    `batch_size` elements; the default behavior is not to drop the smaller
    batch.


In [32]:
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

Now that we have our sequences, we will perform the following steps for each one to create our target text sequences:

1. Grab the input text sequence
2. Assign the target text sequence as the input text sequence shifted by one step forward
3. Group them together as a tuple

In [33]:
def create_seq_targets(seq):
    input_txt = seq[:-1]   #Hello my nam
    target_txt = seq[1:]   #ello my name
    return input_txt, target_txt

In [34]:
dataset = sequences.map(create_seq_targets)

In [36]:
dataset

<MapDataset shapes: ((120,), (120,)), types: (tf.int64, tf.int64)>

In [37]:
for input_txt, target_txt in  dataset.take(1):
    print(input_txt.numpy())
    print(''.join(ind_to_char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    # There is an extra whitespace!
    print(''.join(ind_to_char[target_txt.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


### Generating training batches

Now that we have the actual sequences, we will create the batches, we want to shuffle these sequences into a random order, so the model doesn't overfit to any section of the text, but can instead generate characters given any seed text.

In [38]:
# Batch size
batch_size = 128

# Buffer size to shuffle the dataset so it doesn't attempt to shuffle
# the entire sequence in memory. Instead, it maintains a buffer in which it shuffles elements
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [39]:
dataset

<BatchDataset shapes: ((128, 120), (128, 120)), types: (tf.int64, tf.int64)>

## Step 4: Creating the Model

We will use an LSTM based model with a few extra features, including an embedding layer to start off with and **two** LSTM layers. We based this model architecture off the [DeepMoji](https://deepmoji.mit.edu/) and the original source code can be found [here](https://github.com/bfelbo/DeepMoji).

The embedding layer will serve as the input layer, which essentially creates a lookup table that maps the numbers indices of each character to a vector with "embedding dim" number of dimensions. As you can imagine, the larger this embedding size, the more complex the training. This is similar to the idea behind word2vec, where words are mapped to some n-dimensional space. Embedding before feeding straight into the LSTM usually leads to more realisitic results.

In [40]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
vocab_size

84

In [41]:
# The embedding dimension
embed_dim = 64

# Number of RNN units
rnn_neurons = 1026

Now let's create a function that easily adapts to different variables as shown above.

In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout,GRU

### Setting up Loss Function

For our loss we will use sparse categorical crossentropy, which we can import from Keras. We will also set this as logits=True

In [43]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [ ]:
help(sparse_categorical_crossentropy)

Help on function sparse_categorical_crossentropy in module tensorflow.python.keras.losses:

sparse_categorical_crossentropy(y_true, y_pred, from_logits=False, axis=-1)
    Computes the sparse categorical crossentropy loss.
    
    Standalone usage:
    
    >>> y_true = [1, 2]
    >>> y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
    >>> loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    >>> assert loss.shape == (2,)
    >>> loss.numpy()
    array([0.0513, 2.303], dtype=float32)
    
    Args:
      y_true: Ground truth values.
      y_pred: The predicted values.
      from_logits: Whether `y_pred` is expected to be a logits tensor. By default,
        we assume that `y_pred` encodes a probability distribution.
      axis: (Optional) Defaults to -1. The dimension along which the entropy is
        computed.
    
    Returns:
      Sparse categorical crossentropy loss value.



https://datascience.stackexchange.com/questions/41921/sparse-categorical-crossentropy-vs-categorical-crossentropy-keras-accuracy

In [44]:
def sparse_cat_loss(y_true,y_pred):
    return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [45]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim,batch_input_shape=[batch_size, None]))
    model.add(GRU(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    # Final Dense Layer to Predict
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam', loss=sparse_cat_loss) 
    return model

In [46]:
model = create_model(
  vocab_size = vocab_size,
  embed_dim=embed_dim,
  rnn_neurons=rnn_neurons,
  batch_size=batch_size)

In [47]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 64)           5376      
_________________________________________________________________
gru (GRU)                    (128, None, 1026)         3361176   
_________________________________________________________________
dense (Dense)                (128, None, 84)           86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


## Step 5: Training the model

Let's make sure everything is ok with our model before we spend too much time training! Let's pass in a batch to confirm the model currently predicts random characters without any training.



In [48]:
for input_example_batch, target_example_batch in dataset.take(1):

  # Predict off some random batch
  example_batch_predictions = model(input_example_batch)

  # Display the dimensions of the predictions
  print(example_batch_predictions.shape, " <=== (batch_size, sequence_length, vocab_size)")


(128, 120, 84)  <=== (batch_size, sequence_length, vocab_size)


In [49]:
example_batch_predictions.shape

TensorShape([128, 120, 84])

In [50]:
example_batch_predictions

<tf.Tensor: shape=(128, 120, 84), dtype=float32, numpy=
array([[[ 1.1842721e-03,  8.5755391e-04, -3.3618039e-03, ...,
          2.5133332e-03,  1.1244775e-02,  6.4818356e-03],
        [ 2.6369819e-03,  2.3921486e-04, -4.7738366e-03, ...,
          3.9427388e-03,  1.6698815e-02,  9.1727991e-03],
        [ 6.8185516e-03, -2.3383624e-04, -2.2867264e-03, ...,
         -2.1889564e-03,  1.4886921e-02,  5.2534067e-03],
        ...,
        [ 2.2127554e-03, -7.8938631e-03, -1.3028354e-03, ...,
         -4.0365704e-03, -4.0667886e-03,  4.7552469e-04],
        [-1.0654639e-03, -4.4856570e-03, -2.8850790e-03, ...,
          2.2293692e-03,  2.3970872e-03, -1.1864515e-03],
        [ 3.7032717e-03, -2.5941161e-03,  1.4226725e-03, ...,
         -4.1372771e-03, -1.0428395e-03,  7.6733567e-03]],

       [[-5.7380046e-03, -1.0588712e-03, -1.2653979e-03, ...,
         -9.8620588e-04,  7.8294156e-03,  1.0825766e-03],
        [-2.4442561e-06, -1.6129082e-03,  2.7426914e-03, ...,
         -4.5443452e-03,  1

In [51]:
example_batch_predictions[0].shape

TensorShape([120, 84])

In [52]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

In [53]:
sampled_indices

<tf.Tensor: shape=(120, 1), dtype=int64, numpy=
array([[21],
       [47],
       [17],
       [29],
       [62],
       [66],
       [83],
       [21],
       [70],
       [72],
       [46],
       [31],
       [36],
       [24],
       [ 0],
       [75],
       [79],
       [73],
       [13],
       [70],
       [77],
       [69],
       [26],
       [56],
       [47],
       [77],
       [78],
       [ 0],
       [59],
       [51],
       [25],
       [ 2],
       [55],
       [69],
       [ 7],
       [18],
       [34],
       [24],
       [22],
       [10],
       [29],
       [ 4],
       [29],
       [ 0],
       [25],
       [45],
       [24],
       [33],
       [ 5],
       [28],
       [34],
       [23],
       [46],
       [76],
       [42],
       [27],
       [69],
       [ 4],
       [80],
       [78],
       [ 3],
       [16],
       [28],
       [37],
       [21],
       [35],
       [32],
       [70],
       [14],
       [83],
       [26],
       [47],
       [53],
   

In [54]:
# Reformat to not be a lists of lists
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [55]:
sampled_indices

array([21, 47, 17, 29, 62, 66, 83, 21, 70, 72, 46, 31, 36, 24,  0, 75, 79,
       73, 13, 70, 77, 69, 26, 56, 47, 77, 78,  0, 59, 51, 25,  2, 55, 69,
        7, 18, 34, 24, 22, 10, 29,  4, 29,  0, 25, 45, 24, 33,  5, 28, 34,
       23, 46, 76, 42, 27, 69,  4, 80, 78,  3, 16, 28, 37, 21, 35, 32, 70,
       14, 83, 26, 47, 53, 61, 15, 78, 12, 79,  2, 68, 81, 16, 57, 11, 22,
       50, 47, 24, 41, 50, 29, 20, 68, 35, 56, 73, 54, 12, 53, 35, 76, 79,
       43, 72, 60, 18, 79,  9,  7, 25, 78, 41, 83,  2, 70, 10, 39, 21, 61,
        3])

In [56]:
ind_to_char[sampled_indices]

array([':', 'V', '6', 'D', 'g', 'k', '}', ':', 'o', 'q', 'U', 'F', 'K',
       '>', '\n', 't', 'x', 'r', '2', 'o', 'v', 'n', 'A', 'a', 'V', 'v',
       'w', '\n', 'd', 'Z', '?', '!', '`', 'n', ')', '7', 'I', '>', ';',
       '.', 'D', '&', 'D', '\n', '?', 'T', '>', 'H', "'", 'C', 'I', '<',
       'U', 'u', 'Q', 'B', 'n', '&', 'y', 'w', '"', '5', 'C', 'L', ':',
       'J', 'G', 'o', '3', '}', 'A', 'V', ']', 'f', '4', 'w', '1', 'x',
       '!', 'm', 'z', '5', 'b', '0', ';', 'Y', 'V', '>', 'P', 'Y', 'D',
       '9', 'm', 'J', 'a', 'r', '_', '1', ']', 'J', 'u', 'x', 'R', 'q',
       'e', '7', 'x', '-', ')', '?', 'w', 'P', '}', '!', 'o', '.', 'N',
       ':', 'f', '"'], dtype='<U1')

In [57]:
print("Given the input seq: \n")
print("".join(ind_to_char[input_example_batch[0]]))
print('\n')
print("Next Char Predictions: \n")
print("".join(ind_to_char[sampled_indices ]))


Given the input seq: 

  TOUCHSTONE. 'Thank God.' A good answer.
    Art rich?
  WILLIAM. Faith, sir, so so.
  TOUCHSTONE. 'So so' is good, ver


Next Char Predictions: 

:V6Dgk}:oqUFK>
txr2ovnAaVvw
dZ?!`n)7I>;.D&D
?T>H'CI<UuQBn&yw"5CL:JGo3}AV]f4w1x!mz5b0;YV>PYD9mJar_1]JuxRqe7x-)?wP}!o.N:f"


After confirming the dimensions are working, let's train our network!

In [58]:
epochs = 30

In [59]:
model.fit(dataset,epochs=epochs)

Epoch 1/30
351/351 [==============================] - 41s 118ms/step - loss: 2.5194
Epoch 2/30
351/351 [==============================] - 43s 122ms/step - loss: 1.7184
Epoch 3/30
351/351 [==============================] - 44s 126ms/step - loss: 1.4529
Epoch 4/30
351/351 [==============================] - 45s 127ms/step - loss: 1.3361
Epoch 5/30
351/351 [==============================] - 44s 126ms/step - loss: 1.2740
Epoch 6/30
351/351 [==============================] - 45s 128ms/step - loss: 1.2339
Epoch 7/30
351/351 [==============================] - 44s 126ms/step - loss: 1.2044
Epoch 8/30
351/351 [==============================] - 44s 126ms/step - loss: 1.1807
Epoch 9/30
351/351 [==============================] - 44s 126ms/step - loss: 1.1612
Epoch 10/30
351/351 [==============================] - 45s 127ms/step - loss: 1.1434
Epoch 11/30
351/351 [==============================] - 45s 127ms/step - loss: 1.1276
Epoch 12/30
351/351 [==============================] - 44s 126ms/step - lo

## Step 6: Generating text

Currently our model only expects 128 sequences at a time. We can create a new model that only expects a batch_size=1. We can create a new model with this batch size, then load our saved models weights. Then call .build() on the model:

In [61]:
model.save('/content/drive/My Drive/project_models/shakespeare_gen.h5') 

In [62]:
from tensorflow.keras.models import load_model

In [63]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)

model.load_weights('/content/drive/My Drive/project_models/shakespeare_gen.h5')

model.build(tf.TensorShape([1, None]))


In [64]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 64)             5376      
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1026)           3361176   
_________________________________________________________________
dense_1 (Dense)              (1, None, 84)             86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [65]:
def generate_text(model, start_seed,gen_size=100,temp=1.0):
    '''
    model: Trained Model to Generate Text
    start_seed: Intial Seed text in string form
    gen_size: Number of characters to generate

    Basic idea behind this function is to take in some seed text, format it so
    that it is in the correct shape for our network, then loop the sequence as
    we keep adding our own predicted characters. Similar to our work in the RNN
    time series problems.
    '''

    # Number of characters to generate
    num_generate = gen_size

    # Vecotrizing starting seed text
    input_eval = [char_to_ind[s] for s in start_seed]

    # Expand to match batch format shape
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty list to hold resulting generated text
    text_generated = []

    # Temperature effects randomness in our resulting text
    # The term is derived from entropy/thermodynamics.
    # The temperature is used to effect probability of next characters.
    # Higher probability == lesss surprising/ more expected
    # Lower temperature == more surprising / less expected
 
    temperature = temp

    # Here batch size == 1
    model.reset_states()

    for i in range(num_generate):
        # Generate Predictions
        predictions = model(input_eval)

        # Remove the batch shape dimension
        predictions = tf.squeeze(predictions, 0)

        # Use a cateogircal disitribution to select the next character
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted charracter for the next input
        input_eval = tf.expand_dims([predicted_id], 0)

        # Transform back to character letter
        text_generated.append(ind_to_char[predicted_id])

    return (start_seed + ''.join(text_generated))

In [66]:
print(generate_text(model,"flower",gen_size=1000))

flower's scalb;
    And therefore frame her bosom for a
    man's as a scruple of peak treason fithle. She saids, when he is
    happy made of his pigeops to the world.
  Fal. This inhuman tonts to be.
  Bene. She will stand yourself not- late as love by ta'en
    Upon this I shed in, he wan so sword.  
    This same is she; for highly heart another
    Was sweet lovely from the ground, as now I brought it;
    With these persisting is his value;
    And this right need, what, on vile riding
    Gallia? Shall I take it here! I swear
    As to commit me they will give use them:
    And bidders cannon of the arbour with indeed. Why of the world enjoying!
  CLOWN. You are any which, I protest  
    And see the glassy general shows, take element to thee withal

  PANDARUS. What, ir there to shift?
  GONZALO. Dear sir, do good stragge and undone now in the repukes: to
    old Timon!
     That tert it further; that Polonharr'd you have done.
  SECOND GENTLEMAN. My lord, here he my father, so